In [5]:
pip install cplex

Defaulting to user installation because normal site-packages is not writeable
distutils: /home1/cphillips/.local/lib/python3.9/site-packages
sysconfig: /home1/cphillips/.local/lib64/python3.9/site-packages
user = True
home = None
root = None
prefix = None
Note: you may need to restart the kernel to use updated packages.


In [32]:
import cplex
cpx = cplex.Cplex("example.mps")
cpx.solve()
print("Objective value:", cpx.solution.get_objective_value())
print("Minimizer: x^T =", cpx.solution.get_values())


Selected objective sense:  MINIMIZE
Selected objective  name:  COST
Selected RHS        name:  RHS1
Selected bound      name:  BND1
Version identifier: 22.1.2.0 | 2024-12-10 | f4cec290b
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 1 rows and 1 columns.
Aggregator did 1 substitutions.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.00 ticks)
Objective value: 19.799999999999997
Minimizer: x^T = [0.6000000000000001, 3.6]


In [29]:
import numpy as np
import cplex
from cplex.exceptions import CplexError

def mps_to_standard_form(mps_file):
    try:
        cpx = cplex.Cplex(mps_file)
        cpx.set_results_stream(None)  # mute output

        # Number of variables and constraints
        num_vars = cpx.variables.get_num()
        num_constraints = cpx.linear_constraints.get_num()

        # Objective vector (c)
        c = np.array(cpx.objective.get_linear())

        # Constraint matrix
        A_full = cpx.linear_constraints.get_rows()
        senses = cpx.linear_constraints.get_senses()
        rhs = np.array(cpx.linear_constraints.get_rhs())

        A = []
        G = []
        b = []
        h = []

        for i, (row, sense, rhs_i) in enumerate(zip(A_full, senses, rhs)):
            row_vec = np.zeros(num_vars)
            for idx, val in zip(row.ind, row.val):
                row_vec[idx] = val
            if sense == 'E':  # Equality constraint
                A.append(row_vec)
                b.append(rhs_i)
            elif sense == 'G':  # Greater than or equal
                G.append(row_vec)
                h.append(rhs_i)
            elif sense == 'L':  # Less than or equal
                # convert to -Gx ≥ -h
                G.append(-row_vec)
                h.append(-rhs_i)

        A = np.array(A) if A else np.zeros((0, num_vars))
        b = np.array(b) if b else np.zeros(0)
        G = np.array(G) if G else np.zeros((0, num_vars))
        h = np.array(h) if h else np.zeros(0)

        # Bounds
        l = np.array(cpx.variables.get_lower_bounds())
        u = np.array(cpx.variables.get_upper_bounds())

        c = c.reshape(-1, 1)
        h = h.reshape(-1, 1)
        b = b.reshape(-1, 1)
        l = l.reshape(-1, 1)
        u = u.reshape(-1, 1)
        
        return c, G, h, A, b, l, u

    except CplexError as e:
        print("CPLEX Error:", e)
        return None

In [30]:
import numpy as np

def pdhg(c, G, h, A, b, l, u, max_iter=1000):
    """
    Solves:
        min cᵀx s.t. Gx ≥ h, Ax = b, l ≤ x ≤ u
    using PDHG algorithm.
    """

    # Define Parameters
    eta = 0.9/np.linalg.norm(np.vstack([G,A]), 2)
    omega = 1

    tau = eta/omega
    sigma = eta*omega
    
    m_eq = A.shape[0]
    m_ineq = G.shape[0]
    n = c.shape[0]

    # Initialize Primal and Dual Variables
    x = np.zeros((n, 1))
    y = np.zeros((m_eq, 1))       # for Ax = b
    z = np.zeros((m_ineq, 1))     # for Gx ≥ h → dual z ≥ 0

    for k in range(max_iter):
        
        # Primal update
        x_old = x.copy()
        # Project x onto box constraints l ≤ x ≤ u
        x = np.minimum(np.maximum(x - tau * (c - A.T @ y - G.T @ z), l), u)

        # Dual update
        y += sigma * (b - A @ (2*x - x_old))
        z += sigma * (h - G @ (2*x - x_old))
        z = np.maximum(z, 0)  # project onto constraint z ≥ 0

    print("Objective Value:", (c.T @ x)[0][0])
    return x

In [37]:
c, G, h, A, b, l, u = mps_to_standard_form("example.mps")
print("Minimizer: x^T =",pdhg(c, G, h, A, b, l, u, max_iter=1000000).T[0].tolist())


Selected objective sense:  MINIMIZE
Selected objective  name:  COST
Selected RHS        name:  RHS1
Selected bound      name:  BND1
Objective Value: 19.8
Minimizer: x^T = [0.6000000000000001, 3.6]
